In [1]:
import io
import os
import shutil

output_dir = 'm2l'

In [2]:
SEP = '[sep]'
ALIGN = '[align]'
beam = 5

In [3]:
input_melody_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/data/test.melody'
generate_lyric_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/generate/2021_7_7_m2l_test.log'

generated_file = True
get_last = True
cut_exceed_sent = True

In [4]:
def separate_sentences(x):
    lst = x.copy()
    sep_positions = [i for i,x in enumerate(lst) if x==SEP]
    sep_positions.insert(0,-1)

    ret = []
    for i in range(len(sep_positions)-1):
        sent = lst[sep_positions[i]+1:sep_positions[i+1]] #SZH: not include sep token
        ret.append(sent)
    return ret

In [5]:
def get_songs(file, generated_file = False, get_last = False, cut_exceed_sent=False):
    """
    Get Last : Whether include the last tokens if the sequence not ends with a seperation token
    """
    with io.open(input_melody_file,'r') as f:
        input_melodies= f.readlines()
    input_melodies = list(map(lambda x:x.rstrip('\n').split(' '), input_melodies))
    input_melodies_sent_num = list(map(lambda x:x.count(SEP), input_melodies)) 
    
    with io.open(file,'r') as f:
        lyric_lines = f.readlines()
        if generated_file:
            lyric_lines = list(filter(lambda x:x.startswith('H-'), lyric_lines))
            if len(lyric_lines) == len(input_melodies) * beam:
                lyric_lines.sort( key=lambda x:(int(x.split('\t')[0].split('-')[1]), -float(x.split('\t')[1]) ) )
                lyric_lines = [ x for i,x in enumerate(lyric_lines) if i%beam == 0 ]
            else:    
                lyric_lines.sort( key=lambda x:int(x.split('\t')[0].split('-')[1]) )
    lyric_lines = list(map(lambda x:x.rstrip('\n').split('\t')[-1] ,lyric_lines))    
    
    print(len(lyric_lines), len(input_melodies))
    assert len(lyric_lines)==len(input_melodies)
    
    lyric_seqs = list(map(lambda x:x.rstrip('\n').split(' '),lyric_lines))
    lyric_seqs = [ch for ch in lyric_seqs if ch != ALIGN]
    
    
    if get_last:
        for i in range(len(lyric_seqs)):
            if lyric_seqs[i][-1] != SEP:
                lyric_seqs[i].append(SEP)
    
    # 分句子
    lyric_seq_sents = list(map(separate_sentences, lyric_seqs))
    
   
    return_list = []
    for i,sent_seq in enumerate(lyric_seq_sents):
        if cut_exceed_sent and len(sent_seq) > input_melodies_sent_num[i]:
            sent_seq = sent_seq[0:input_melodies_sent_num[i]]
        cur_song_return = []

        for j,sent in enumerate(sent_seq):
            cur_song_return.extend(sent)
            cur_song_return.append(SEP)
        return_list.append(cur_song_return)
    return return_list

In [6]:
output_lyrics = get_songs(generate_lyric_file, generated_file, get_last, cut_exceed_sent)

714 714


In [7]:
with io.open(input_melody_file,'r') as f:
    input_melodies= f.readlines()
input_melodies = list(map(lambda x:x.rstrip('\n').split(' '), input_melodies))
input_melodies_sent_num = list(map(lambda x:x.count(SEP), input_melodies))

output_lyrics_sent_num = list(map(lambda x:x.count(SEP), output_lyrics))
list(zip(input_melodies_sent_num, output_lyrics_sent_num))

[(22, 22),
 (25, 25),
 (21, 21),
 (4, 4),
 (32, 32),
 (28, 28),
 (6, 6),
 (8, 8),
 (9, 9),
 (23, 23),
 (15, 15),
 (3, 3),
 (23, 23),
 (6, 6),
 (10, 10),
 (4, 4),
 (20, 20),
 (10, 10),
 (18, 18),
 (1, 1),
 (3, 3),
 (32, 32),
 (18, 18),
 (7, 7),
 (1, 1),
 (1, 1),
 (29, 29),
 (31, 31),
 (21, 21),
 (9, 9),
 (6, 6),
 (20, 20),
 (30, 30),
 (15, 15),
 (8, 8),
 (5, 5),
 (7, 7),
 (19, 19),
 (19, 19),
 (19, 19),
 (5, 5),
 (30, 30),
 (13, 13),
 (28, 28),
 (12, 12),
 (12, 12),
 (32, 32),
 (5, 5),
 (24, 24),
 (4, 4),
 (10, 10),
 (20, 20),
 (27, 27),
 (29, 29),
 (13, 13),
 (20, 20),
 (20, 20),
 (20, 20),
 (23, 23),
 (1, 1),
 (27, 27),
 (28, 28),
 (16, 16),
 (2, 2),
 (22, 22),
 (12, 12),
 (4, 4),
 (26, 26),
 (20, 20),
 (20, 20),
 (1, 1),
 (23, 23),
 (10, 10),
 (7, 7),
 (18, 18),
 (16, 16),
 (35, 35),
 (40, 40),
 (9, 9),
 (20, 20),
 (30, 30),
 (3, 3),
 (7, 7),
 (27, 27),
 (2, 2),
 (3, 3),
 (1, 1),
 (16, 16),
 (21, 21),
 (11, 11),
 (27, 27),
 (19, 19),
 (7, 7),
 (5, 5),
 (11, 11),
 (8, 8),
 (23, 23),
 

In [11]:
x[1]

['you',
 'can',
 'lead',
 'me',
 'to',
 'know',
 '[sep]',
 'a',
 'friend',
 'of',
 'pride',
 '[sep]',
 'you',
 'will',
 'never',
 'never',
 'know',
 '[sep]',
 'take',
 'your',
 'brother',
 '[sep]',
 'a',
 'daydream',
 '[sep]',
 'and',
 'i',
 'want',
 'to',
 'real',
 'close',
 'your',
 'eyes',
 'and',
 '[sep]',
 'feel',
 'the',
 'pain',
 '[sep]',
 'you',
 'will',
 'show',
 'me',
 'so',
 'extreme',
 '[sep]',
 'or',
 'touch',
 'me',
 'dear',
 'for',
 'real',
 '[sep]',
 'i',
 'think',
 'of',
 'your',
 'life',
 '[sep]',
 'oh',
 'i',
 'love',
 'you',
 '[sep]',
 'and',
 'no',
 'no',
 '[sep]',
 'i',
 'shall',
 'make',
 'it',
 'hard',
 'to',
 'sing',
 '[sep]',
 'a',
 'record',
 'you',
 'like',
 '[sep]',
 'and',
 '[sep]',
 'to',
 'keep',
 'your',
 'love',
 '[sep]',
 'i',
 'will',
 'always',
 'be',
 'the',
 'last',
 'star',
 '[sep]',
 'you',
 'strength',
 '[sep]',
 'and',
 'keep',
 'your',
 'din',
 '[sep]',
 'of',
 'two',
 'minds',
 'and',
 'waiting',
 '[sep]',
 'who',
 'is',
 'wild',
 'and',
 't

In [8]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

In [13]:
for i,x in enumerate(zip(input_melodies,output_lyrics)):
    cur_folder = os.path.join(output_dir, str(i))
    os.makedirs(cur_folder)
    with io.open(os.path.join(cur_folder,'train.melody'),'w') as f:
        f.write(" ".join(x[0]))
    with io.open(os.path.join(cur_folder,'train.lyric'),'w') as f:
        f.write(" ".join(x[1]))